In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process 
import datetime as dt

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
dhc = pd.read_csv('DHC_data/DHC_data.csv')
wslive = pd.read_csv('Data/wslive_with_results_2019-06-07_to_2020-02-21.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4,10,16,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
inc_2020 = wslive[(wslive['new_metric']=='Inconclusive')&(wslive['ws_year']==2020)]

In [25]:
inc_2020 = inc_2020[inc_2020.OFFICE_ADDRESS_LINE_2.isna()==False]
inc_2020 = inc_2020.drop_duplicates('OFFICE_TELEPHONE')

In [26]:
inc = inc_2020[['PHYSICIAN_ME_NUMBER', 'PHYSICIAN_FIRST_NAME', 'PHYSICIAN_MIDDLE_NAME',
       'PHYSICIAN_LAST_NAME', 'SUFFIX', 'DEGREE', 'OFFICE_ADDRESS_LINE_1',
       'OFFICE_ADDRESS_LINE_2', 'OFFICE_ADDRESS_CITY', 'OFFICE_ADDRESS_STATE',
       'OFFICE_ADDRESS_ZIP', 'OFFICE_TELEPHONE', 'WSLIVE_FILE_DT']]

In [27]:
dhc = dhc.rename(columns ={'Additional Office Locations':'Add_Offices', 'Phone Number':'Phone_Number', 'Zip Code':'Zipcode_DHC', 'First Name':'First', 'Physician Name':'Name_DHC',
       'Middle Name':'Middle', 'Primary Specialty':'Spec', 'Primary Hospital Affiliation':'Affiliation',
       'Last Name':'Last'})

In [28]:
dhc_phone = pd.merge(dhc, inc, left_on='ME',right_on='PHYSICIAN_ME_NUMBER')

In [29]:
len(dhc_phone)

937

In [31]:
dhc_phone = dhc_phone.fillna('None')
phones=[]
for row in dhc_phone.itertuples():
    phones.append(row.Phone_Number.replace('.',''))
dhc_phone.Phone_Number=phones

In [32]:
def add_match(df):
    count = 0
    for row in df.itertuples():
        match = False
        if row.OFFICE_ADDRESS_LINE_2 in row.Address.upper():
            match = True
        if row.Address.upper() in row.OFFICE_ADDRESS_LINE_2:
            match = True
        if fuzz.ratio(row.OFFICE_ADDRESS_LINE_2,row.Address.upper())>70:
            match = True
        if match == True:
            count +=1
    return count

def phone_match(df):
    count = 0
    for row in df.itertuples():
        if row.Phone_Number == str(int(row.OFFICE_TELEPHONE)):
            count+=1
    return count

def zip_match(df):
    count = 0
    for row in df.itertuples():
        if row.Zipcode_DHC == row.OFFICE_ADDRESS_ZIP:
            count+=1
    return count

def state_match(df):
    count = 0
    for row in df.itertuples():
        if row.State == row.OFFICE_ADDRESS_STATE:
            count+=1
    return count

def city_match(df):
    count = 0
    for row in df.itertuples():
        if row.City.upper() == row.OFFICE_ADDRESS_CITY:
            count+=1
    return count

In [33]:
def return_counts(add_df):
    p =phone_match(add_df)
    a =add_match(add_df)
    z =zip_match(add_df)
    c =city_match(add_df)
    s = state_match(add_df)
    print(f'{p} ({round(p/len(add_df)*100,2)}%) phones match')
    print(f'{a} ({round(a/len(add_df)*100,2)}%) addresses match')
    print(f'{z} ({round(z/len(add_df)*100,2)}%) zipcodes match')
    print(f'{c} ({round(c/len(add_df)*100,2)}%) cities match')
    print(f'{s} ({round(s/len(add_df)*100,2)}%) states match')

In [34]:
# samp = dhc_phone.sample(500)
return_counts(dhc_phone)

316 (33.72%) phones match
496 (52.93%) addresses match
474 (50.59%) zipcodes match
600 (64.03%) cities match
857 (91.46%) states match


In [132]:
len(dhc_address)

4799

In [100]:
good_phone.groupby(['ws_year','ws_month']).count()

PHYSICIAN_ME_NUMBER  PHYSICIAN_FIRST_NAME  \
ws_year ws_month                                              
2019    6                       13178                 13178   
        7                       12021                 12021   
        8                       12549                 12549   
        9                        8659                  8659   
        10                      10347                 10346   
        11                       9761                  9760   
        12                       8935                  8935   
2020    1                        6919                  6918   
        2                        6360                  6360   

                  PHYSICIAN_MIDDLE_NAME  PHYSICIAN_LAST_NAME  SUFFIX  DEGREE  \
ws_year ws_month                                                               
2019    6                         11124                13178     425       0   
        7                         10133                12021     393       0   
        8                         10397                12549     385       0   
        9                          7190                 8658     258       0   
        10                         8510                10347     314       0   
        11                         7933                 9761     262       0   
        12                         7246                 8935     249       0   
2020    1                          5732                 6919     205       0   
        2                          5373                 6360     234       0   

                  OFFICE_ADDRESS_LINE_1  OFFICE_ADDRESS_LINE_2  \
ws_year ws_month                                                 
2019    6                          7328                  13178   
        7                          4229                  12021   
        8                          4453                  12549   
        9                          2925                   8659   
        10                         2944                  10347   
        11                         2851                   9760   
        12                         2600                   8935   
2020    1                          2233                   6919   
        2                          1772                   6360   

                  OFFICE_ADDRESS_CITY  OFFICE_ADDRESS_STATE  ...  \
ws_year ws_month                                             ...   
2019    6                       13178                 13178  ...   
        7                       12021                 12021  ...   
        8                       12549                 12549  ...   
        9                        8659                  8659  ...   
        10                      10347                 10347  ...   
        11                       9759                  9759  ...   
        12                       8932                  8932  ...   
2020    1                        6919                  6919  ...   
        2                        6360                  6360  ...   

                  MATCH_ADDR_LONG  SPECIALTY  SPECIALTY_UPDATED  \
ws_year ws_month                                                  
2019    6                       0      13178              13178   
        7                       0      12021              12021   
        8                       0      12549              12549   
        9                       0       8659               8659   
        10                      0      10347              10347   
        11                      0       9761               9761   
        12                      0       8935               8935   
2020    1                       0       6919               6919   
        2                       0       6360               6360   

                  SOURCE_FILE_DT  new_metric  addr_status  pe_status  \
ws_year ws_month                                                       
2019    6                      0       13178        13178      13178   
        7    

,PHYSICIAN_ME_NUMBER,PHYSICIAN_FIRST_NAME,PHYSICIAN_MIDDLE_NAME,PHYSICIAN_LAST_NAME,SUFFIX,DEGREE,OFFICE_ADDRESS_LINE_1,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,...,SPECIALTY_UPDATED,SOURCE_FILE_DT,ws_month,ws_year,new_metric,addr_status,pe_status,phone_status,fax_status,spec_status
299577,2.640691e+10,YURI,EDUARDO,CORRALES,NaN,NaN,URB COUNTRY CLUB,820 AVE ITURREGUI,SAN JUAN,PR,...,1.0,2019-12-19,2,2020,Updated,Confirmed,Updated,Confirmed,Confirmed,Confirmed
299578,2.512102e+09,CASEY,CATHERINE,SAGER,NaN,NaN,NaN,3409 LUDINGTON ST STE 204,ESCANABA,MI,...,1.0,2019-12-19,2,2020,Updated,Confirmed,Confirmed,Confirmed,Updated,Confirmed
299579,1.102932e+09,CATHERINE,C,MADAFFARI,NaN,NaN,COMM INTERNAL MED,4500 SAN PABLO RD S,JACKSONVILLE,FL,...,1.0,2019-12-19,2,2020,Updated,Confirmed,Updated,Confirmed,Confirmed,Confirmed
299580,2.401021e+09,CATHERINE,PAGE,CROSLAND,NaN,NaN,UNITY HEALTH CARE INC,425 2ND ST NW,WASHINGTON,DC,...,1.0,2019-12-19,2,2020,Updated,Confirmed,Confirmed,Confirmed,Confirmed,Confirmed
299581,4.301000e+08,CAROLYN,M,BOLTIN,NaN,NaN,NaN,525 E 68TH ST,NEW YORK,NY,...,1.0,2019-12-19,2,2020,Updated,Confirmed,Confirmed,Updated,Updated,Confirmed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310150,4.950295e+10,ROOPALI,NaN,AGGARWAL,NaN,NaN,NaN,1 WEXFORD DR,MONMOUTH JUNCTION,NJ,...,1.0,2020-01-27,2,2020,Updated,Updated,Updated,Confirmed,Confirmed,Confirmed
310151,5.104770e+09,ROBERT,CHILES,ALVIS,NaN,NaN,NaN,3468 PARK AVE,BRONX,NY,...,1.0,2020-01-27,2,2020,Updated,Updated,Updated,Confirmed,Updated,Confirmed
310152,7.029603e+08,PATRICK,CHARLES,BARRY,NaN,NaN,NaN,45 COUNTY ROAD 804 STE 210,FRASER,CO,...,1.0,2020-01-27,2,2020,Updated,Updated,Confirmed,Confirmed,Confirmed,Confirmed
310153,3.050101e+10,ROHIT,NaN,KHIRBAT,NaN,NaN,NaN,9901 MEDICAL CENTER DR,ROCKVILLE,MD,...,1.0,2020-01-27,2,2020,Updated,Updated,Updated,Confirmed,Updated,Confirmed


In [134]:
good_phone.WS

,PHYSICIAN_ME_NUMBER,PHYSICIAN_FIRST_NAME,PHYSICIAN_MIDDLE_NAME,PHYSICIAN_LAST_NAME,SUFFIX,DEGREE,OFFICE_ADDRESS_LINE_1,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,...,SPECIALTY_UPDATED,SOURCE_FILE_DT,ws_month,ws_year,new_metric,addr_status,pe_status,phone_status,fax_status,spec_status
0,1.002122e+09,TRAVIS,JOHN,MAIERS,NaN,NaN,NaN,1 CAPITAL WAY,PENNINGTON,NJ,...,1.0,NaN,6,2019,Updated,Confirmed,Updated,Confirmed,Updated,Confirmed
1,1.222090e+09,WILLIAM,HENRY,MCRAE,III,NaN,STE 1400,471 E BROAD ST,COLUMBUS,OH,...,1.0,NaN,6,2019,Updated,Confirmed,Confirmed,Confirmed,Updated,Confirmed
2,1.606101e+09,WHITNEY,ELIZABETH,ZIRKLE,NaN,NaN,STE 1325,675 N SAINT CLAIR ST,CHICAGO,IL,...,1.0,NaN,6,2019,Updated,Confirmed,Updated,Confirmed,Updated,Confirmed
3,1.643631e+09,WALTER,CHAS,WROBEL,NaN,NaN,NaN,623 E MAUDE AVE,ARLINGTON HEIGHTS,IL,...,1.0,NaN,6,2019,Updated,Confirmed,Updated,Confirmed,Updated,Confirmed
4,2.002112e+09,SUSAN,BERSOT,VINCI,NaN,NaN,NaN,200 ABRAHAM FLEXNER WAY,LOUISVILLE,KY,...,1.0,NaN,6,2019,Updated,Confirmed,Updated,Confirmed,Updated,Confirmed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310150,4.950295e+10,ROOPALI,NaN,AGGARWAL,NaN,NaN,NaN,1 WEXFORD DR,MONMOUTH JUNCTION,NJ,...,1.0,2020-01-27,2,2020,Updated,Updated,Updated,Confirmed,Confirmed,Confirmed
310151,5.104770e+09,ROBERT,CHILES,ALVIS,NaN,NaN,NaN,3468 PARK AVE,BRONX,NY,...,1.0,2020-01-27,2,2020,Updated,Updated,Updated,Confirmed,Updated,Confirmed
310152,7.029603e+08,PATRICK,CHARLES,BARRY,NaN,NaN,NaN,45 COUNTY ROAD 804 STE 210,FRASER,CO,...,1.0,2020-01-27,2,2020,Updated,Updated,Confirmed,Confirmed,Confirmed,Confirmed
310153,3.050101e+10,ROHIT,NaN,KHIRBAT,NaN,NaN,NaN,9901 MEDICAL CENTER DR,ROCKVILLE,MD,...,1.0,2020-01-27,2,2020,Updated,Updated,Updated,Confirmed,Updated,Confirmed


In [142]:
GUTE = good_phone[good_phone.WSLIVE_FILE_DT=='2020-02-14'].drop_duplicates('OFFICE_TELEPHONE')

In [46]:
dhc_phone[dhc_phone.Phone_Number!='None'].sample(500).to_csv('DHC_MD_Inconclusive_Sample.csv',index=False)

In [160]:
dhc_phone = dhc_phone.fillna('None')

In [45]:
dhc_phone[(dhc_phone['Phone_Number']!='None')]

,NPI,ME,Add_Offices,Phone_Number,Zipcode_DHC,State,City,Address,First,Name_DHC,...,PHYSICIAN_LAST_NAME,SUFFIX,DEGREE,OFFICE_ADDRESS_LINE_1,OFFICE_ADDRESS_LINE_2,OFFICE_ADDRESS_CITY,OFFICE_ADDRESS_STATE,OFFICE_ADDRESS_ZIP,OFFICE_TELEPHONE,WSLIVE_FILE_DT
0,1467762740,5.121004e+08,None,9169837561,95630,CA,Folsom,1650 Creekside Dr,Matthew,Matthew Storment,...,STORMENT,None,None,FOLSOM ANESTHESIA MEDICAL GROUP,1650 CREEKSIDE DR,FOLSOM,CA,95630,2.259267e+09,2020-02-14
1,1518256585,4.901110e+09,"Oregon City, OR",5036921212,97062,OR,Tualatin,19300 Sw 65th Ave,Jacob,Jacob Adams,...,ADAMS,None,None,ROC ORTHOPEDICS,1508 DIVISION ST STE 105,OREGON CITY,OR,97045,4.153536e+09,2020-01-17
2,1528346756,1.803112e+09,"Madison, WI",6177260604,"=""02114""",MA,Boston,55 Fruit St,David,David Hennessy,...,HENNESSY,None,None,None,55 FRUIT ST,BOSTON,MA,02114,6.177249e+09,2020-01-10
3,1538322250,1.643081e+09,None,5206948888,85724,AZ,Tucson,1501 N Campbell Ave,Richard,Richard Amini,...,AMINI,None,None,None,1501 N CAMPBELL AVE,TUCSON,AZ,85724,5.206940e+09,2020-01-31
4,1447546171,3.050110e+10,None,8138550111,33626,FL,Tampa,11327 Countryway Blvd,Tali,Tali Arviv,...,ARVIV,None,None,None,11327 COUNTRYWAY BLVD,TAMPA,FL,33626,8.889124e+09,2020-01-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932,1902973886,2.604871e+09,None,8777497428,37404,TN,Chattanooga,2333 Mccallie Ave,Paula,Paula Bugay,...,STEWART,None,None,SISKIN PHYSICAL MEDICINE & REHABILITATIO,1 SISKIN PLZ STE 101,CHATTANOOGA,TN,37403,6.158486e+09,2020-01-24
933,1013174143,2.437697e+10,None,3182218411,71101,LA,Shreveport,510 E Stoner Ave,Peimin,Peimin Zhu,...,ZHU,None,None,None,1541 KINGS HWY,SHREVEPORT,LA,71103,3.186755e+09,2020-02-21
934,1033116306,3.080183e+10,None,7877537123,"=""00918""",PR,San Juan,550 Ave Domenech,Leonardo J,Leonardo J Campiz Jimenez,...,CAMPIZ,None,None,URB EL SENORIAL,285 AVE WINSTON CHURCHILL,RIO PIEDRAS,PR,00926,7.877537e+09,2020-01-10
935,1891012449,5.606101e+09,None,2628966000,53186,WI,Waukesha,W231N1440 Corporate Ct,Kyle,Kyle Nelson,...,NELSON,None,None,None,W231N1440 CORPORATE CT,WAUKESHA,WI,53186,8.604473e+09,2020-02-14


In [170]:
samp = dhc_phone.sample(500)

In [179]:
DHC_PHONE = dhc_phone[['NPI', 'ME', 'Add_Offices', 'Phone_Number', 'Zipcode_DHC', 'State',
       'City', 'Address', 'First', 'Name_DHC', 'Middle', 'Spec', 'Affiliation',
       'Last', 'Address1', 'PHYSICIAN_ME_NUMBER', 'PHYSICIAN_FIRST_NAME',
       'PHYSICIAN_MIDDLE_NAME', 'PHYSICIAN_LAST_NAME', 'SUFFIX', 'DEGREE',
       'OFFICE_ADDRESS_LINE_1', 'OFFICE_ADDRESS_LINE_2', 'OFFICE_ADDRESS_CITY',
       'OFFICE_ADDRESS_STATE', 'OFFICE_ADDRESS_ZIP','OFFICE_TELEPHONE', 'WSLIVE_SOURCE',
       'SPECIALTY']]

In [185]:
sample = DHC_PHONE.sample(500)

In [184]:
DHC_PHONE = DHC_PHONE.drop_duplicates('Phone_Number')

In [186]:
sample.to_csv('Data/dhc_phone_sample.csv', index=False)